In [1]:
import pandas as pd, numpy as np

In [2]:
seed = 1
np.random.seed(seed)

In [3]:
%matplotlib inline

## Load data

In [4]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
train_df.shape, test_df.shape

((4209, 378), (4209, 377))

In [5]:
train_df.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test_df.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [7]:
train_df.ID.max()

8417

In [8]:
test_df.ID.max()

8416

# preprocessing

In [9]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
# train_input = train_df.drop("ID") First, experiment with keeping id
train_input = train_df.copy()
test_input = test_df.copy()

In [12]:
num_train =len( train_input)
num_train

4209

In [13]:
full_input = pd.concat([train_input, test_input])
full_input.shape

(8418, 378)

In [14]:
categorical_cols = full_input.select_dtypes(include=[np.object]).columns.tolist()
categorical_cols

['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']

In [15]:
full_input = pd.get_dummies(full_input)

In [43]:
train_input = full_input[full_input.y.notnull()]
test_input = full_input[full_input.y.isnull()]
test_input.drop('y', axis=1, inplace=True)
print train_input.shape, test_input.shape
train_input.head()

(4209, 581) (4209, 581)


,ID,X10,X100,X101,X102,X103,X104,X105,X106,X107,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,9,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
t = [['a','b']]
t.append(['hi'])
t.append(['hey', 'a'])
t += [['hu', 'g']]
t

[['a', 'b'], ['hi'], ['hey', 'a'], ['hu', 'g']]

In [97]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, full_dataset, look_back=5, is_train=True, pad_val = -1):
    
    dataX, dataY = [], []
    
    if is_train:
        dropcols = ["ID", "y"]
    else:
        dropcols = ["ID"]
    
    for index, row in dataset.iterrows():
        print 'is train: ' + str(is_train) + ': ' + str(index) + '\r',
        current_id = row.ID
        last_n_previous_rows = full_dataset[full_dataset['ID'] < current_id].drop(dropcols, axis=1).tail(look_back).values
        #last_n_previous_rows = pad_sequences([last_n_previous_rows], maxlen=look_back, padding='post', truncating='post', value=pad_val)[0]
        x = last_n_previous_rows.tolist()
        x.append(row.drop(dropcols).values.tolist())
        dataX.append(x)
        if is_train:
            dataY.append(row.y)
        
    if is_train:
        return dataX, dataY
    else:
        return dataX

In [99]:
train_dataset_x, train_dataset_y = create_dataset(train_input, full_input, look_back=5, is_train=True, pad_val = -1)
test_dataset_x = create_dataset(test_input, full_input, look_back=5, is_train=False, pad_val = -1)
np.array(train_dataset_x).shape, np.array(train_dataset_y).shape, np.array(test_dataset_x).shape

((4209,), (4209,), (4209,))

In [107]:
train_dataset_x_padded = pad_sequences(train_dataset_x, maxlen=6, padding='pre', truncating='pre', value=-1)
test_dataset_x_padded = pad_sequences(test_dataset_x, maxlen=6, padding='pre', truncating='pre', value=-1)

ValueError: cannot convert float NaN to integer

In [120]:
np.where(np.array(test_dataset_x[3]) == np.NaN)

(array([], dtype=int64), array([], dtype=int64))

In [126]:
train_dataset_y = np.array(train_dataset_y)

## model

In [141]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, Masking
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import Bidirectional
from keras import backend as K

In [124]:
from keras_tqdm import TQDMNotebookCallback

In [142]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [146]:
model = Sequential()
model.add(Masking(mask_value=-1, input_shape=(5 + 1, 579)))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='relu'))

model.compile(loss='mse', optimizer='adam', metrics=[r2_keras])

In [147]:
model.fit(train_dataset_x_padded, 
          train_dataset_y, 
          epochs=200, 
          validation_split=0.2,
          shuffle=True,
          verbose=2)

Train on 3367 samples, validate on 842 samples
Epoch 1/200
45s - loss: 7689.0397 - r2_keras: -5.3958e+01 - val_loss: 5271.8173 - val_r2_keras: -4.2566e+01
Epoch 2/200
4s - loss: 3946.5084 - r2_keras: -2.6507e+01 - val_loss: 2009.6310 - val_r2_keras: -1.5535e+01
Epoch 3/200
5s - loss: 1519.8551 - r2_keras: -9.3897e+00 - val_loss: 471.6571 - val_r2_keras: -2.8142e+00
Epoch 4/200
6s - loss: 784.3577 - r2_keras: -4.3673e+00 - val_loss: 173.4830 - val_r2_keras: -3.6227e-01
Epoch 5/200
5s - loss: 717.4120 - r2_keras: -3.9739e+00 - val_loss: 145.6674 - val_r2_keras: -1.3679e-01
Epoch 6/200
5s - loss: 701.6817 - r2_keras: -3.9650e+00 - val_loss: 137.7748 - val_r2_keras: -7.3827e-02
Epoch 7/200
6s - loss: 701.1729 - r2_keras: -3.9468e+00 - val_loss: 140.0201 - val_r2_keras: -9.1618e-02
Epoch 8/200
8s - loss: 682.1189 - r2_keras: -3.6570e+00 - val_loss: 143.4256 - val_r2_keras: -1.1880e-01
Epoch 9/200
8s - loss: 722.1145 - r2_keras: -3.9262e+00 - val_loss: 141.7374 - val_r2_keras: -1.0530e-01
Ep

6s - loss: 505.5726 - r2_keras: -2.5337e+00 - val_loss: 47.7678 - val_r2_keras: 0.6346
Epoch 81/200
6s - loss: 507.3315 - r2_keras: -2.5875e+00 - val_loss: 47.7992 - val_r2_keras: 0.6330
Epoch 82/200
6s - loss: 484.2725 - r2_keras: -2.3499e+00 - val_loss: 64.8897 - val_r2_keras: 0.4901
Epoch 83/200
6s - loss: 490.2745 - r2_keras: -2.3375e+00 - val_loss: 54.2948 - val_r2_keras: 0.5799
Epoch 84/200
6s - loss: 462.4170 - r2_keras: -2.2985e+00 - val_loss: 48.0762 - val_r2_keras: 0.6292
Epoch 85/200
6s - loss: 489.5366 - r2_keras: -2.3528e+00 - val_loss: 47.3577 - val_r2_keras: 0.6346
Epoch 86/200
6s - loss: 477.6763 - r2_keras: -2.3217e+00 - val_loss: 51.9801 - val_r2_keras: 0.5998
Epoch 87/200
6s - loss: 471.4435 - r2_keras: -2.2386e+00 - val_loss: 49.2821 - val_r2_keras: 0.6218
Epoch 88/200
7s - loss: 487.8778 - r2_keras: -2.2511e+00 - val_loss: 50.4240 - val_r2_keras: 0.6111
Epoch 89/200
6s - loss: 503.6014 - r2_keras: -2.5004e+00 - val_loss: 49.9199 - val_r2_keras: 0.6148
Epoch 90/200


KeyboardInterrupt: 

## Kaggle predictions

In [77]:
kaggle_preds = gbm.predict(test_input.values)
kaggle_preds.shape

(4209,)

In [79]:
test_df['y'] = kaggle_preds
test_df[['ID', 'y']].head()

,ID,y
0,1,79.069446
1,2,95.031323
2,3,78.065370
3,4,78.209953
4,5,115.343554


In [80]:
test_df[['ID', 'y']].to_csv("../data/submission_lightgbm_benchmark.csv", index=False)